In [1]:
import numpy as np
import py3Dmol as p3d
import veloxchem as vlx

In [2]:
methanol_ff = vlx.Molecule.read_smiles("CO")

In [3]:
xtb_drv = vlx.XtbDriver()

xtb_opt_drv = vlx.OptimizationDriver(xtb_drv)
xtb_opt_drv.ostream.mute()

xtb_opt_results = xtb_opt_drv.compute(methanol_ff)

methanol_xtb = vlx.Molecule.read_xyz_string(xtb_opt_results["final_geometry"])

In [4]:
basis = vlx.MolecularBasis.read(methanol_xtb, "STO-3G", ostream=None)

scf_drv = vlx.ScfRestrictedDriver()
scf_drv.ostream.mute()
scf_results = scf_drv.compute(methanol_xtb, basis)

opt_drv = vlx.OptimizationDriver(scf_drv)
opt_drv.ostream.mute()
methanol_hf = opt_drv.compute(methanol_xtb, basis, scf_results)

In [5]:
viewer = p3d.view(viewergrid=(1, 3), width=500, height=200, linked=False)

viewer.addModel(methanol_ff.get_xyz_string(), "xyz", viewer=(0, 0))
viewer.addModel(methanol_xtb.get_xyz_string(), "xyz", viewer=(0, 1))
viewer.addModel(methanol_hf["final_geometry"], "xyz", viewer=(0, 2))

viewer.setViewStyle({"style": "outline", "width": 0.05})
viewer.setStyle({"stick": {}, "sphere": {"scale": 0.25}})

viewer.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [6]:
#print("From force field (UFF):")
#dm_ff = methanol_ff.get_distance_matrix_in_angstrom()
#print(np.around(dm_ff, 3))
#print()

#print("From xTB:")
#dm_xtb = methanol_xtb.get_distance_matrix_in_angstrom()
#print(np.around(dm_xtb, 3))
#print()

#print("From HF:")
#dm_hf = methanol_hf.get_distance_matrix_in_angstrom()
#print(np.around(dm_hf, 3))

In [7]:
#print("Difference between HF and xTB:")
#print(np.around(dm_hf - dm_xtb, 3))
#print()
#print("Maximal absolute difference between HF and xTB:")
#print(f"{np.max(np.abs(dm_hf - dm_xtb)):.4f} Å")

In [8]:
# replace the diagonal elements (zeros) with a large number to avoid nan
#dm_tmp = dm_hf + 1e10 * np.diag(np.ones(len(dm_hf)))

#print("Relative difference between HF and xTB:")
#print(np.around((dm_hf - dm_xtb) / dm_tmp, 3))
#print()
#print("Maximal relative difference between HF and xTB:")
#print(f"{np.max(np.abs((dm_hf - dm_xtb)/dm_tmp)):.4f}")